# Gemini API: Prompting with Video

This notebook provides a quick example of how to prompt Gemini 1.5 Pro using a video file. In this case, you'll use a short clip of [Big Buck Bunny](https://peach.blender.org/about/).

In [1]:
!pip install -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.2/164.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.3/718.3 kB 23.5 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.

In [2]:
import google.generativeai as genai

### Authentication Overview

**Important:** The File API uses API keys for authentication and access. Uploaded files are associated with the API key's cloud project. Unlike other Gemini APIs that use API keys, your API key also grants access data you've uploaded to the File API, so take extra care in keeping your API key secure. For best practices on securing API keys, refer to Google's [documentation](https://support.google.com/googleapi/answer/6310037).

### Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.

In [4]:
import os
from google.colab import userdata

os.environ['GOOGLE_API_KEY']=userdata.get('gemini_key')

genai.configure()

## Upload a video to the Files API

The Gemini API accepts video file formats directly. The File API accepts files under 2GB in size and can store up to 20GB of files per project. Files last for 2 days and cannot be downloaded from the API. For the example, you will use the short film "Big Buck Bunny".

> "Big Buck Bunny" is (c) copyright 2008, Blender Foundation / www.bigbuckbunny.org and [licensed](https://peach.blender.org/about/) under the [Creative Commons Attribution 3.0](http://creativecommons.org/licenses/by/3.0/) License.

Note: You can also [upload other files](https://download.blender.org/peach/bigbuckbunny_movies/) to use.

In [5]:
!wget https://download.blender.org/peach/bigbuckbunny_movies/BigBuckBunny_320x180.mp4

--2024-07-21 08:31:03--  https://download.blender.org/peach/bigbuckbunny_movies/BigBuckBunny_320x180.mp4
Resolving download.blender.org (download.blender.org)... 104.22.64.163, 104.22.65.163, 172.67.14.163, ...
Connecting to download.blender.org (download.blender.org)|104.22.64.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64657027 (62M) [video/mp4]
Saving to: ‘BigBuckBunny_320x180.mp4’

BigBuckBunny_320x18 100%[===================>]  61.66M  43.2MB/s    in 1.4s    

2024-07-21 08:31:05 (43.2 MB/s) - ‘BigBuckBunny_320x180.mp4’ saved [64657027/64657027]



In [6]:
video_file_name = "BigBuckBunny_320x180.mp4"

print(f"Uploading file...")
video_file = genai.upload_file(path=video_file_name)
print(f"Completed upload: {video_file.uri}")

Uploading file...
Completed upload: https://generativelanguage.googleapis.com/v1beta/files/xsql26vq2a3o


## Get File

After uploading the file, you can verify the API has successfully received the files by calling `files.get`.

`files.get` lets you see the file uploaded to the File API that are associated with the Cloud project your API key belongs to. Only the `name` (and by extension, the `uri`) are unique.

In [7]:
video_file.state.name

'PROCESSING'

In [8]:
import time

while video_file.state.name == "PROCESSING":
    print('Waiting for video to be processed.')
    time.sleep(10)
    video_file = genai.get_file(video_file.name)

if video_file.state.name == "FAILED":
  raise ValueError(video_file.state.name)
print(f'Video processing complete: ' + video_file.uri)

Waiting for video to be processed.
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/xsql26vq2a3o


## Generate Content

After the video has been uploaded, you can make `GenerateContent` requests that reference the File API URI.

In [9]:
# Create the prompt.
prompt = "Describe this video."

# Set the model to Gemini 1.5 flash.
model = genai.GenerativeModel(model_name="models/gemini-1.5-flash-latest")

# Make the LLM request.
print("Making LLM inference request...")
response = model.generate_content([prompt, video_file],
                                  request_options={"timeout": 600})
print(response.text)

Making LLM inference request...
The video starts with a scene of a lush green field with trees and a blue sky. It transitions to a shot of a small bird sitting on a branch. Then, the words "The Peach Open Movie Project Presents" appear on the screen. After that, a large, gray rabbit emerges from a hole in a tree. The rabbit yawns and stretches. The title "Big Buck Bunny" appears in white text over the scene. The rabbit walks towards a large tree and sits down underneath it. A squirrel is seen climbing the tree, while a chinchilla sits near the base of the tree. The chinchilla is holding a nut in its paws. 

The rabbit is seen interacting with the squirrels, who are trying to steal his apple. The rabbit gets angry and uses a vine to tie the squirrels to the tree. 

The video ends with a shot of the rabbit standing in the field. 



In [10]:
from IPython.display import Markdown

Markdown(response.text)

The video starts with a scene of a lush green field with trees and a blue sky. It transitions to a shot of a small bird sitting on a branch. Then, the words "The Peach Open Movie Project Presents" appear on the screen. After that, a large, gray rabbit emerges from a hole in a tree. The rabbit yawns and stretches. The title "Big Buck Bunny" appears in white text over the scene. The rabbit walks towards a large tree and sits down underneath it. A squirrel is seen climbing the tree, while a chinchilla sits near the base of the tree. The chinchilla is holding a nut in its paws. 

The rabbit is seen interacting with the squirrels, who are trying to steal his apple. The rabbit gets angry and uses a vine to tie the squirrels to the tree. 

The video ends with a shot of the rabbit standing in the field. 


## Delete File

Files are automatically deleted after 2 days or you can manually delete them using `files.delete()`.

In [ ]:
genai.delete_file(video_file.name)
print(f'Deleted file {video_file.uri}')

Deleted file https://generativelanguage.googleapis.com/v1beta/files/2poj7kqlvop3


## Learning more

The File API lets you upload a variety of multimodal MIME types, including images, audio, and video formats. The File API handles inputs that can be used to generate content with [`model.generateContent`](https://ai.google.dev/api/rest/v1/models/generateContent) or [`model.streamGenerateContent`](https://ai.google.dev/api/rest/v1/models/streamGenerateContent).

The File API accepts files under 2GB in size and can store up to 20GB of files per project. Files last for 2 days and cannot be downloaded from the API.

* Learn more about the [File API](https://github.com/google-gemini/cookbook/blob/main/quickstarts/File_API.ipynb) with the quickstart.

* Learn more about prompting with [media files](https://ai.google.dev/tutorials/prompting_with_media) in the docs, including the supported formats and maximum length.